<a href="https://colab.research.google.com/github/vmpreciado/NETS3120/blob/main/TraditionalML4LinkPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we illustrate the use of Machine Learning (ML) for link suggestions in the Facebook (FB) graph. We will follow a traditional ML approach that involves manually selecting features, creating a model based on logistic regression, training it, and then evaluating its performance. Let's start by loading the Facebook graph:

In [1]:
# Import necessary libraries
from google.colab import drive
import networkx as nx
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Mount Google Drive: This is used if you decided to work in Colab using files in your Google Drive
drive.mount('/content/drive')

# Path to your file on Google Drive: 1) Download facebook_combined.txt.gz; 2) Decompressed as a .txt; 3) Rename and locate wherever you like...
file_path = '/content/drive/My Drive/ColabNotebooks/FacebookSmall.txt'

# Create a new graph and name it FB
FB = nx.Graph()# Create a graph from the edgelist file
FB = nx.read_edgelist(file_path, create_using=nx.Graph(), nodetype=int)

# Perform initial checks: Make sure the number of nodes and edges are correct...
print(f"Number of nodes: {FB.number_of_nodes()}")
print(f"Number of edges: {FB.number_of_edges()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of nodes: 4039
Number of edges: 88234


First, we need to create features for each pair of nodes that can be used to predict whether a link should exist between them. Some common features for link prediction in social networks include: # of Common Neighbors, Jaccard Coefficient, or
the product of the degrees of the two nodes.

In [2]:
# Feature Engineering
def compute_features(pair):
    u, v = pair
    common_neighbors = len(sorted(nx.common_neighbors(FB, u, v)))
    jaccard_coefficient = list(nx.jaccard_coefficient(FB, [pair]))[0][2]
    preferential_attachment = list(nx.preferential_attachment(FB, [pair]))[0][2] # this index is equal to the product of the degrees

    features = {
        "common_neighbors": common_neighbors,
        "jaccard_coefficient": jaccard_coefficient,
        "preferential_attachment": preferential_attachment,
    }
    return features

Second, for supervised learning, we need to construct a dataset with examples of connected pair of nodes (positive examples) and disconnected pair of nodes (negative examples). We can use existing links in your graph as positive examples. For negative examples, we can randomly sample pairs of nodes that are not connected. Because the graph is sparse, there are many more negative examples that positive examples. This issue is called 'Class Imbalance' and it can dramatically hurt the performance of our logistic classifier. To overcome this issue, we subsample the number of negative examples so that we have as many negative examples as positive examples.

In [3]:
# Constructing the Dataset
positive_samples = [(u, v) for u, v in FB.edges()]
negative_samples = [(u, v) for u, v in nx.non_edges(FB) if not FB.has_edge(u, v)]
negative_samples = random.sample(negative_samples, len(positive_samples)) # Balancing classes with randomization

# Compute features for all samples
all_samples = positive_samples + negative_samples
labels = [1] * len(positive_samples) + [0] * len(negative_samples)
features = [compute_features(pair) for pair in all_samples]

# Convert to DataFrame
df = pd.DataFrame(features)
df['label'] = labels

Let's take a look at the dataframe...

In [4]:
print(df.head()) # This line shows the first 5 (positive) samples
print(df.tail()) # This line shows the last 5 (negative) samples
(f"Number of samples in each class: {len(positive_samples)}")

   common_neighbors  jaccard_coefficient  preferential_attachment  label
0                16             0.045977                     5899      1
1                 9             0.025862                     3470      1
2                16             0.045977                     5899      1
3                 9             0.025862                     3470      1
4                12             0.034483                     4511      1
        common_neighbors  jaccard_coefficient  preferential_attachment  label
176463                 1             0.013333                     1044      0
176464                 0             0.000000                      196      0
176465                 0             0.000000                      120      0
176466                 0             0.000000                      234      0
176467                 4             0.117647                      280      0


'Number of samples in each class: 88234'

Third, let's split our dataset into training and test sets. A common split ratio is 50% training and 50% testing (since we are not going to do model selection, we do not need a validation subset). We use the training set to train our model and the test set to evaluate the model's performance.

In [5]:
# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.3, random_state=42)

Since we are dealing with a binary classification problem (link exists or not), we can use a simple Logistic Regression.

In [6]:
# Model Selection and Training
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

Once our model is trained, we can evaluate our model's performance on the test set using metrics such as accuracy, precision, recall, F1 score, or ROC AUC. This will give you an idea of how well your model can predict new links in the network.

In [7]:
# Evaluation
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy:", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96     26548
           1       0.98      0.93      0.96     26393

    accuracy                           0.96     52941
   macro avg       0.96      0.96      0.96     52941
weighted avg       0.96      0.96      0.96     52941

Accuracy: 0.9568387450180389


Once trained and evaluated, we can use our model to score potential new links in the network. For a given user, we can predict the likelihood of a link for all other users they are not currently connected to and suggest the links with the highest scores.

Remember, the quality of your predictions heavily depends on the features you choose and the balance of your dataset. You should experiment with different features to find the best approach for your specific network.